Set up your GitHub personal access token, repository, file path, and commit message

In [16]:
import requests
import base64
import json

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
REPO_OWNER = 'rtr-ai'
REPO_NAME = 'aia-rag'
FILE_PATH = 'verordnung-2024-1689_final_version.txt'
COMMIT_MESSAGE = 'Updated file content using Python script'
BRANCH_NAME = 'bojan'

# Set headers for authentication
headers = {
    'Authorization': f'Bearer {GITHUB_TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

Get the file's current SHA and content

In [ ]:
file_url = f'https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/contents/data/{FILE_PATH}?ref={BRANCH_NAME}'
response = requests.get(file_url, headers=headers)
response_data = response.json()

file_sha = response_data['sha']
file_content = base64.b64decode(response_data['content']).decode('utf-8')
print("Current file content:", file_content)

Remove empty lines

In [ ]:
# Create a list to store non-empty lines
non_empty_lines = []

for line in file_content.splitlines():
    if line.strip():  # Check if the line is not empty after stripping whitespace
        non_empty_lines.append(line)

# Join non-empty lines and print the result
new_text = '\n'.join(non_empty_lines)
print("New text output:\n", new_text)

Adding #, ## and ### to KAPITEL. ABSCHNITTand Artikel, respectively. 

In [ ]:
# Split the formatted text into lines
lines = new_text.strip().split('\n')

# Create a new list to store the updated lines with headings
updated_lines = []

# Iterate over each line and add the appropriate heading
for line in lines:
    if line.startswith("KAPITEL"):
        updated_lines.append(f"# {line}")
    elif line.startswith("ABSCHNITT"):
        updated_lines.append(f"## {line}")
    elif line.startswith("Artikel"):
        updated_lines.append(f"### {line}")
    else:
        updated_lines.append(line)  # Keep other lines unchanged

# Join the updated lines into the final output
new_text = "\n".join(updated_lines)
print(new_text)

Add a colon after each instance of 'Kapitel,' 'Abschnitt,' and 'Artikel,' and append the subtitle to each accordingly.

In [22]:
import re

new_text = re.sub(
    r"(# KAPITEL\s+[IVXLCDM]+|## ABSCHNITT\s+\d+|ANHANG\s+[IVXLCDM]+)\n([^\n]+)|(### \bArtikel\s+\d+)\n([^\n]+)",
    lambda m: f"{m.group(1) or m.group(3)}: {m.group(2) or m.group(4)}",
    new_text
)

Fetching text in one line

In [23]:
new_text = re.sub(r'\((\d+)\)\s*\n', r'(\1) ', new_text) # (176)
                                                         # some text 
new_text = re.sub(r'([a-z])\)\s*\n', r'\1) ', new_text)  # a)
                                                         # some text 
new_text = re.sub(r'(\d+)\.\s*\n', r'\1. ', new_text)    # 51.
                                                         # some text 

In [25]:
new_content = file_content.replace(file_content, new_text)
encoded_content = base64.b64encode(new_content.encode('utf-8')).decode('utf-8')

Update the file with new content

In [26]:
data = {
    'message': COMMIT_MESSAGE,
    'content': encoded_content,
    'sha': file_sha,
    'branch': BRANCH_NAME
}

update_response = requests.put(file_url, headers=headers, data=json.dumps(data))

# Confirm success
if update_response.status_code == 200:
    print("File updated successfully")
else:
    print("Error updating file:", update_response.json())


File updated successfully
